## Bibliotecas

In [1]:
import random
import math

import pandas as pd

from functools import partial
from scipy.stats import t, chi2

## Códigos auxiliares

In [2]:
from utils import *

## Variáveis globais

In [3]:
# N_RODADAS = 3200
N_RODADAS = 32 

# N_AMOSTRAS = 10000
N_AMOSTRAS = 10

RHOS = [0.2, 0.4, 0.6, 0.8, 0.9]

CHEGADA = 1
FIM_DO_SERVICO = 0

MU = 1.0

LIVRE = 0
OCUPADO = 1

FCFS = 1
LCFS = 0

## Funcionamento geral

### Gerando VAs exponenciais
Resultado mostrado na aula 5, slide 5.

In [4]:
def gera_exponencial(rho):

    # Gera VA uniforme no intervalo [0.0, 1.0)
    u0 = random.random()

    # Pega amostra da exponencial
    x0 = -math.log(u0)/ rho

    return x0

### Gerando números aleatórios
Os seeds em Python são definidos por meio da função random.seed(int) e fixamos um seed antes cada simulação para garantir a reprodutibilidade dos resultados. Para garantir a independência dos seeds, rodamos o algoritmo com alguns seeds diferentes ao longo do trabalho.

Além disso também fizemos alguns testes para mostrar que não há sobreposição entre os intervalos gerados pelos seeds.

Mesmo usando seeds próximas (0 e 1, por exemplo) não encontramos sobreposição das sequências de valores nem uma correlação significativa. Acreditamos que isso se deve ao fato de o Python utilizar o algoritmo do Mersenne Twister para gerar números pseudoaleatórios (https://en.wikipedia.org/wiki/Mersenne_Twister).

In [5]:
def gera_amostras(rho, seed):

    rodadas = []

    random.seed(seed)
    
    for _ in range(N_RODADAS):
        amostras = []

        for _ in range(N_AMOSTRAS):
            amostras.append(gera_exponencial(rho))
        
        rodadas.append(amostras)
    
    # Lista de rodadas com números aleatórios gerados pela amostra exponencial
    return rodadas

#### Teste 1

Primeiro geramos 2 vetores com váriaveis exponenciais com mesma taxa, cada um a partir de uma seed diferente. E depois verificamos se existem valores em comuns nesses vetores.

In [6]:
def teste_gera_amostras(rodadas_A, rodadas_B):

    tamanho_lista_rodadas = len(rodadas_A)

    for rodada_A in range(tamanho_lista_rodadas - 1):
        lista_rodada = rodadas_A[rodada_A]

        for rodada_B in range(tamanho_lista_rodadas):
            verifica_lista = rodadas_B[rodada_B]

            verificacao = list(set(lista_rodada).intersection(verifica_lista))
            if(len(verificacao) > 0):
                return "Foram encontrados valores iguais"
            
    return "Não há valores iguais entre as duas rodadas"       

Exemplo:

In [7]:
rodadas_A = gera_amostras(0.3, 0)
rodadas_B = gera_amostras(0.3, 1)

teste_gera_amostras(rodadas_A, rodadas_B)

'Não há valores iguais entre as duas rodadas'

#### Teste 2

No outro teste geramos 2 vetores também e verificamos o indíce de correlação (Pearson) entre eles.

- +1: correlação positiva completa
- +0.8: correlação positiva forte
- +0.6: correlação positiva moderada
- 0: nenhuma correlação
- -0.6: correlação negativa moderada
- -0.8: correlação negativa forte
- -1: correlação negativa completa

Referência: https://stackabuse.com/calculating-pearson-correlation-coefficient-in-python-with-numpy/


In [8]:
def teste_pearson_gera_amostras(seed_1, seed_2):

    rodadas = []
    amostras_1 = []
    amostras_2 = []

    random.seed(seed_1)
    for _ in range(N_AMOSTRAS):
        amostras_1.append(random.random())
    rodadas.append(amostras_1)

    random.seed(seed_2)
    for _ in range(N_AMOSTRAS):
        amostras_2.append(random.random())
    rodadas.append(amostras_2)

    x_pearson = pd.Series(rodadas[0])
    y_pearson = pd.Series(rodadas[1])

    resultado = x_pearson.corr(y_pearson)

    return resultado

Exemplo:

In [9]:
teste_pearson_gera_amostras(0, 1)

-0.03549463742990013

### Eventos

In [10]:
class Evento:

    def __init__(self, tipo_evento, instante_t, id_cliente):
        self.tipo_evento = tipo_evento
        self.instante_t = instante_t
        self.id_cliente = id_cliente

    # Define como a classe é impressa
    def __repr__(self):
        if(self.tipo_evento == CHEGADA):
            escreve_string = "CHEGADA"
        else:
            escreve_string = "FIM DO SERVIÇO"

        return f"Tipo de evento: {escreve_string}\n Instante: {self.instante_t}\n ID Cliente: {self.id_cliente}"

Exemplo:

In [11]:
evento = Evento(CHEGADA, 0.1, 0)
evento

Tipo de evento: CHEGADA
 Instante: 0.1
 ID Cliente: 0

### Clientes

In [12]:
class Cliente:

    def __init__(self, chegada_evento, cor):
        self.id = chegada_evento.id_cliente
        self.tempo_chegada = chegada_evento.instante_t
        self.tempo_espera = 0.0
        self.cor = cor

    # Define como a classe é impressa
    def __repr__(self):
        return f"ID: {self.id}\n Tempo de chegada: {self.tempo_chegada}\n Tempo de espera: {self.tempo_espera}"

In [13]:
def cria_cliente(evento, cor):
    novo_cliente = Cliente(evento, cor)
    return novo_cliente

Exemplo:

In [14]:
cor = "%06x" % random.randint(0, 0xFFFFFF)
cliente = cria_cliente(evento, cor)
cliente

ID: 0
 Tempo de chegada: 0.1
 Tempo de espera: 0.0

### Gerando chegadas

In [15]:
def gera_chegadas(instante_t, distribuicao_chegada, id_cliente):
    chegada = Evento(CHEGADA, instante_t + distribuicao_chegada(), id_cliente)
    return chegada

Exemplo:

In [32]:
chegada = gera_chegadas(1, partial(gera_exponencial, 0.3), 0)
chegada

Tipo de evento: CHEGADA
 Instante: 6.021559177248312
 ID Cliente: 0

### Gerando fins de serviço

In [35]:
def gera_fim_servico(instante_t, distribuicao_servico, id_cliente):
    fim_servico = Evento(FIM_DO_SERVICO, instante_t + distribuicao_servico(), id_cliente)
    return fim_servico

Exemplo:

In [36]:
fim_servico = gera_fim_servico(10, partial(gera_exponencial, 0.3), 1)
fim_servico

Tipo de evento: FIM DO SERVIÇO
 Instante: 12.338526565082633
 ID Cliente: 1

### Lista de eventos

### Fila de clientes

### Serviço

### Intervalos de confiança

#### T-Student
Resultado apresentado na aula 7, slides 4 e 6.

In [19]:
t_student_percentil = t(df = N_AMOSTRAS - 1).ppf((0.025, 0.975))[1]

def t_student(media, variancia):

    # Verifica se a média e variância têm valores nulos (ou próximos disso)
    if(math.isclose(media, 0, abs_tol = 1e-9)):
        media = 0

    if(math.isclose(variancia, 0, abs_tol = 1e-9)):
        variancia = 0

    # Se a média e variância forem nulas
    if(not(media or variancia)):
        return 0.0, 0.0, 0.0

    desvio_padrao = math.sqrt(variancia)

    # Metade do intervalo
    metade = t_student_percentil * (desvio_padrao/ math.sqrt(N_AMOSTRAS))

    # Limite superior do intervalo
    limite_superior = media + metade

    # Limite inferior do intervalo
    limite_inferior = media - metade

    # Precisão do intervalo
    precisao = metade/ media

    return limite_superior, limite_inferior, precisao

#### Chi-Quadrado
Resultado apresentado na aula 7, slides 12 ao 14.

In [20]:
chi2_0025_percentil, chi2_0975_percentil = chi2(df = N_AMOSTRAS - 1).ppf((0.025, 0.975))

def chi_quadrado(variancia):

    if(variancia == 0.0):
        return 0.0, 0.0, 0.0

    else:
        limite_superior = (N_AMOSTRAS) * variancia/ chi2_0975_percentil
        limite_inferior = (N_AMOSTRAS) * variancia/ chi2_0025_percentil

        # Enunciado pede que seja aproximadamente 0.05
        precisao = (chi2_0975_percentil - chi2_0025_percentil)/ (chi2_0975_percentil + chi2_0025_percentil)

        return limite_superior, limite_inferior, precisao

### Estimadores

Essa classe permitey o cálculo iterativo da média e da variãncia de uma variável, recebendo uma amostra de cada vez.
Esse resultado foi apresentado na aula 7, slide 19.

In [21]:
class Estimador:

    def __init__(self):
        self.soma_amostras = 0.0
        self.quadrado_soma_amostras = 0.0
        self.numero_amostras = 0

    def adiciona_amostra(self, amostra):
        self.soma_amostras += amostra
        self.quadrado_soma_amostras += (amostra ** 2)
        self.numero_amostras += 1

    def media(self):
        return self.soma_amostras/ self.numero_amostras

    def variancia(self):
        termo_auxiliar_1 = self.quadrado_soma_amostras/ (self.numero_amostras - 1)
        termo_auxiliar_2 = (self.soma_amostras ** 2)/ (self.numero_amostras * (self.numero_amostras - 1))
        return termo_auxiliar_1 - termo_auxiliar_2

    def t_student(self):
        return t_student(self.media(), self.variancia())

    def variancia_chi_quadrado(self):
        return chi_quadrado(self.variancia())

    def chi_quadrado(self):
        return chi_quadrado(self.media())

### Simulação

In [84]:
random.seed(42) #setando uma seed pra manter os testes iguais

fila1 = []
fila2 = []
tempo = 0
eventos = [ gera_chegadas(0, partial(gera_exponencial, 0.3), 0) ] # começa populando com uma chegada

i = 1
while i < 20: #loop de eventos, chegada de 10 clientes
    atual = eventos.pop(0)
    tempo = atual.instante_t
    print("Tempo", tempo)
    
    if atual.tipo_evento == CHEGADA: #evento de chegada
        fila1.append( cria_cliente(atual, cor) ) #adiciona o cliente chegado na fila
        eventos.append(gera_chegadas(tempo, partial(gera_exponencial, 0.3), i) ) #cria o próximo evento de chegada
        print("Chegada", fila1[-1])
        
        checa_termino = any([e.tipo_evento == FIM_DO_SERVICO for e in eventos])
        if not checa_termino: #essa foi a primeira chegada (não tem pessoas na fila 1)
            eventos.append( gera_fim_servico(tempo, partial(gera_exponencial, 0.3), atual.id_cliente) ) #adiciona o próximo evento de término
        i+= 1
    
    elif atual.tipo_evento == FIM_DO_SERVICO:
        cliente = fila1.pop(0) #remove o primeiro da fila
        cliente.tempo_espera = tempo - cliente.tempo_chegada #atualiza o tempo q esperou
        
        if len(fila1) != 0: #se a fila não estiver vazia
            eventos.append( gera_fim_servico(tempo, partial(gera_exponencial, 0.3), fila1[0].id) ) #cria evento de término para o próximo da fila
        print("Saída", cliente)
        
    eventos.sort(key = lambda e: e.instante_t)
    print(eventos)
    print()
#eventos

Tempo 1.4906104378394698
Chegada ID: 0
 Tempo de chegada: 1.4906104378394698
 Tempo de espera: 0.0
[Tipo de evento: FIM DO SERVIÇO
 Instante: 5.7935356869958206
 ID Cliente: 0, Tipo de evento: CHEGADA
 Instante: 13.78544156357473
 ID Cliente: 1]

Tempo 5.7935356869958206
Saída ID: 0
 Tempo de chegada: 1.4906104378394698
 Tempo de espera: 4.302925249156351
[Tipo de evento: CHEGADA
 Instante: 13.78544156357473
 ID Cliente: 1]

Tempo 13.78544156357473
Chegada ID: 1
 Tempo de chegada: 13.78544156357473
 Tempo de espera: 0.0
[Tipo de evento: FIM DO SERVIÇO
 Instante: 14.805058658512774
 ID Cliente: 1, Tipo de evento: CHEGADA
 Instante: 18.784238029062944
 ID Cliente: 2]

Tempo 14.805058658512774
Saída ID: 1
 Tempo de chegada: 13.78544156357473
 Tempo de espera: 1.0196170949380434
[Tipo de evento: CHEGADA
 Instante: 18.784238029062944
 ID Cliente: 2]

Tempo 18.784238029062944
Chegada ID: 2
 Tempo de chegada: 18.784238029062944
 Tempo de espera: 0.0
[Tipo de evento: FIM DO SERVIÇO
 Instante: 

## Testes de correção

### Corretude do simulador

## Estimativa da fase transiente